## Import libraries

In [1]:
import ee
import geemap

## Create an interactive map

In [2]:
Map = geemap.Map(center=[40, -100], zoom=4)

## Add Earth Engine Python script

In [3]:
# Add Earth Engine dataset
image = ee.Image("USGS/SRTMGL1_003")

#  ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#  Chapter:      F5.1 Raster/Vector Conversions
#  Checkpoint:   F51a
#  Authors:      Keiko Nomura, Samuel Bowers
#  ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

#-------------#
# Section 1.1 #
#-------------#

# Load raster (elevation) and vector (colombia) datasets.
elevation = ee.Image('USGS/GMTED2010').rename('elevation')
colombia = ee.FeatureCollection(
'FAO/GAUL_SIMPLIFIED_500m/2015/level0') \
.filter(ee.Filter.equals('ADM0_NAME', 'Colombia'))

# Display elevation image.
Map.centerObject(colombia, 7)
Map.addLayer(elevation, {
    'min': 0,
    'max': 4000
}, 'Elevation')

# Initialize image with zeros and define elevation zones.
zones = ee.Image(0) \
.where(elevation.gt(100), 1) \
.where(elevation.gt(200), 2) \
.where(elevation.gt(500), 3)

# Mask pixels below sea level (<= 0 m) to retain only land areas.
# Name the band with values 0-3 as 'zone'.
zones = zones.updateMask(elevation.gt(0)).rename('zone')

Map.addLayer(zones, {
    'min': 0,
    'max': 3,
    'palette': ['white', 'yellow', 'lime', 'green'],
    'opacity': 0.7
}, 'Elevation zones')

projection = elevation.projection()
scale = elevation.projection().nominalScale()

elevationVector = zones.reduceToVectors(
geometry = colombia.geometry(),
crs = projection,
scale = 1000, 
geometryType = 'polygon',
eightConnected = False,
labelProperty = 'zone',
bestEffort = False,
maxPixels = 1e13,
tileScale = 3 
)

print(elevationVector.limit(10).getInfo())

elevationDrawn = elevationVector.draw(
color = 'black',
strokeWidth = 1
)
Map.addLayer(elevationDrawn, {}, 'Elevation zone polygon')

zonesSmooth = zones.focalMode(4, 'square')

zonesSmooth = zonesSmooth.reproject(projection.atScale(scale))

Map.addLayer(zonesSmooth, {
    'min': 0,
    'max': 3,
    'palette': ['white', 'yellow', 'lime', 'green'],
    'opacity': 0.7
}, 'Elevation zones (smooth)')

elevationVectorSmooth = zonesSmooth.reduceToVectors(
geometry = colombia.geometry(),
crs = projection,
scale = scale,
geometryType = 'polygon',
eightConnected = False,
labelProperty = 'zone',
bestEffort = False,
maxPixels = 1e13,
tileScale = 3
)

smoothDrawn = elevationVectorSmooth.draw(
color = 'black',
strokeWidth = 1
)
Map.addLayer(smoothDrawn, {}, 'Elevation zone polygon (smooth)')

#-------------#
# Section 1.2 #
#-------------#

geometry = ee.Geometry.Polygon([
[-89.553, -0.929],
[-89.436, -0.929],
[-89.436, -0.866],
[-89.553, -0.866],
[-89.553, -0.929]
])

# To zoom into the area, un-comment and run below
# Map.centerObject(geometry,12)
Map.addLayer(geometry, {}, 'Areas to extract points')

elevationSamples = elevation.sample(
region = geometry,
projection = projection,
scale = scale,
geometries = True,
)

Map.addLayer(elevationSamples, {}, 'Points extracted')

# Add three properties to the output table:
# 'Elevation', 'Longitude', and 'Latitude'.

def func_jeh(feature):
    geom = feature.geometry().coordinates()
    return ee.Feature(None, {
            'Elevation': ee.Number(feature.get(
            'elevation')),
            'Long': ee.Number(geom.get(0)),
            'Lat': ee.Number(geom.get(1))
        })

elevationSamples = elevationSamples.map(func_jeh)









# Export as CSV.
geemap.ee_export_vector_to_drive(
collection = elevationSamples,
description = 'extracted_points',
fileFormat = 'CSV'
)

elevationSamplesStratified = zones.stratifiedSample(
numPoints = 10,
classBand = 'zone',
region = geometry,
scale = scale,
projection = projection,
geometries = True
)

Map.addLayer(elevationSamplesStratified, {}, 'Stratified samples')

#  -----------------------------------------------------------------------
#  CHECKPOINT
#  -----------------------------------------------------------------------

{'type': 'FeatureCollection', 'columns': {'count': 'Long<0, 4294967295>', 'system:index': 'String', 'zone': 'Byte<0, 3>'}, 'features': [{'type': 'Feature', 'geometry': {'geodesic': False, 'type': 'Polygon', 'coordinates': [[[-81.73342995905446, 12.475998189514755], [-81.72444680621327, 12.475998189514755], [-81.72444680621327, 12.484981342355951], [-81.71546365337207, 12.484981342355951], [-81.71546365337207, 12.51193080087954], [-81.70648050053089, 12.51193080087954], [-81.70648050053089, 12.538880259403115], [-81.69749734768969, 12.538880259403115], [-81.69749734768969, 12.5748128707679], [-81.6795310420073, 12.5748128707679], [-81.6795310420073, 12.583796023609096], [-81.6885141948485, 12.583796023609096], [-81.6885141948485, 12.592779176450293], [-81.71546365337207, 12.592779176450293], [-81.71546365337207, 12.583796023609096], [-81.72444680621327, 12.583796023609096], [-81.72444680621327, 12.556846565085507], [-81.73342995905446, 12.556846565085507], [-81.73342995905446, 12.529897

## Display the interactive map

In [4]:
Map

Map(center=[3.8953576070659985, -73.07893922598336], controls=(WidgetControl(options=['position', 'transparent…